
<font size="5">**<center> ĐỒ ÁN 01 </center>** </font>

<font size="4">**<center> Thu nhập dữ liệu & EDA </center>** </font>

> Họ tên: Thái Bá Sơn

> MSSV: 1712732

# I/ Dữ liệu

### Thông tin về dữ liệu:

Dữ liệu này cung cấp thành tích học tập và các thông tin liên quan của học sinh trung học ở 2 trường của Bồ Đào Nha. Cụ thể các thuộc tính dữ liệu bao gồm điểm tổng kết, yếu tố tác động như tình trạng, hoàn cảnh gia đình... được thu thập bằng cách sử dụng thông tin mà nhà trường cung cấp và phiếu khảo sát từ các học sinh.  

**Nguồn:** dữ liệu được lấy [tại đây](https://www.kaggle.com/dipam7/student-grade-prediction)

# II/ Khám phá dữ liệu:

## 1/ Thu thập dữ liệu:

### Import

In [22]:
pip install soundcloud

  Created wheel for soundcloud: filename=soundcloud-0.5.0-py3-none-any.whl size=12545 sha256=824e7410ca9626325295f0618091d9bd2be09a782a86ab44d3f2606e21fe3550
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\b5\c6\7a\b568cd85b47dd88abcea5461cf14677905625cdcbc4ffcbee1
  Created wheel for fudge: filename=fudge-1.1.1-py3-none-any.whl size=33910 sha256=cf2577266fc295f75b68ace9ddc1a6006d678fda0c395e9cc36846cfb3cccc8d
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\54\d9\66\b00600b4853459997d283c9e5e9f011760cb795e768c35a962
Successfully built soundcloud fudge
Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re
import soundcloud
import logging
import random
#Import thư viện:
from requests_html import HTMLSession, HTML
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

### a/ API:

Link mẫu thông số API: https://stackoverflow.com/questions/18632072/how-to-get-total-number-of-tracks-in-a-playlist-using-soundcloud-json-api

Link mẫu ID format: https://developers.soundcloud.com/docs/api/reference#tracks

In [50]:
#pip install soundcloud-lib

  Created wheel for soundcloud-lib: filename=soundcloud_lib-0.5.4-py3-none-any.whl size=8216 sha256=384443932e2eef876336ca182c701acbd7c8f574f2f20ff5feaa053f71a51954
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\e3\3e\87\9cc85e4017ad772cd3b9c725d5d5a5fce64515a066c3d088a2
Successfully built soundcloud-lib
Note: you may need to restart the kernel to use updated packages.


In [84]:
from sclib import SoundcloudAPI, Track, Playlist

api = SoundcloudAPI()  # never pass a Soundcloud client ID that did not come from this library

# playlist = api.resolve('https://soundcloud.com/s-n-th-i-6/sets/sitatc')
playlist = api.resolve('https://soundcloud.com/yeonkkot/sets/s12')
for track in playlist.tracks:
    print("{} - {}\n".format(track.artist, track.title))

noel s - lift

Richard Griffith - Midnight Walk

Gorila - Akaido

Joey Pecoraro - Partly Sunny

Tom Misch & Carmody - Paper Planes

HONNE - Woman

Kupla - fell in love in a dream

Gagle - Love Note (Original Instrumental)

Amy Winehouse - Rehab (Tom Misch Remix)

EDEN - Fumes (feat. gnash)

Tom Misch - Dawsons Heights

Kendall Miles - Her (feat. a l e x)

Lazy Lewis - White Winds

j'san. - under the trees

Jaiza Trio - Chucho (Paquito D'Rivera)

saib. - Sakura Trees

DeKobe - Raining

tom holmes - Evening Jazz 3 (live)

jack cc & 626 - j bags

Kupla - still breathing

Matt Simons - Catch & Release (Deepend Remix) - [OUT NOW!!]

Lazy Lewis - Diamond Dreams

Trazzhnef Complications 2econd - Sanity Canopy

prima - see you soon.

Slowz - Holding You (Stan Sax Edit)

Lazy Lewis - Dancing Days

Simon Jefferis - George Lassos the Moon

Yaoundé (OGSE - Olivier Gay Small Ensemble)

Stephen Onprachanh - Paradise (Demo)

tom holmes - Afternoon Jazz Beatz (Chillhop)

Aso - My (Now on Spotify)

Aso

Link: https://stackoverflow.com/questions/40992480/getting-a-soundcloud-api-client-id

### b/ parse HTML:

- Sử dụng thư viện selenium tương tác với trang chủ Soundcloud để lấy thông tin về link các playlist:

In [2]:
# Mở Chrome và cho full màn hình
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.maximize_window()

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/86.0.4240.22/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\dell\.wdm\drivers\chromedriver\win32\86.0.4240.22]


In [3]:
#Tải trang cho đến hết:
def load_link(browser, link):
    #Vào link:
    browser.get(link)
    
    #Xóa quảng cáo:
    dismiss = browser.find_elements_by_class_name('announcement__dismiss')
    dismiss[0].click()
    
    #Cuộn trang
    SCROLL_PAUSE_TIME = 3

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [23]:
# link_browser = 'https://soundcloud.com/discover'

# #Mở trang chủ của Soundcloud và load trang: 
# load_link(browser, link_browser)

# #Chờ trang tải:
# time.sleep(4)

# #load toàn bộ trang để hiện nhiều playlist nhất 
# buttons = browser.find_elements_by_class_name('tileGallery__sliderPeekForward')
# t = 0
# l = len(buttons)
# while t < l: 
#     if buttons[t].is_displayed():
#         buttons[t].click()
#     else:
#         browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         t+=1

In [24]:
# # Xuất link tất cả playlist
# playlists = browser.find_elements_by_class_name('audibleTile__audibleHeading')
# print("Số link playlist thu được: ", len(playlists))

# #Xuất kết quả vào file link_playlists.txt
# f = open('link_playlists.txt', 'w')
# for playlist in playlists:
#     if playlist.get_attribute('href'):
#         f.write("{}\n".format(playlist.get_attribute('href')))
# f.close()


Số link playlist thu được:  300


### Lọc thông tin về track từ danh sách link playlist:

- Thu thập dữ liệu từ playlist gợi ý ở trang chủ Soundcloud:

In [5]:
#Lưu thông tin playlist vào file:
def file_write(filename, track_list, track_link_list, user_name_list, user_link_list, playcount_list, duration_list):
    with open(filename, 'a', encoding='utf-8') as f:
        for i in range(len(track_list)):
            f.write(f'{track_list[i]}\t{track_link_list[i]}\t{user_name_list[i]}\t{user_link_list[i]}\t{playcount_list[i]}\t{duration_list[i]}\n')
        f.close()

In [6]:
#Hàm tương tác với trang để lấy thông tin và lưu vào file:
def get_playlist_info(playlist_file, track_file, browser, link):
    #Tải hết trang:
    load_link(browser, link)
    
    time.sleep(3)
    browser.find_element_by_tag_name('html').send_keys(Keys.ESCAPE)
    
    PAUSE_TIME = 0.2
    
    #List:
    track_list = []
    track_link_list = []
    user_name_list = []
    user_link_list = []
    playcount_list = []
    duration_list = []
    empty = []
    
    time.sleep(2)
    #Tìm tên track
    tracks = browser.find_elements_by_class_name('trackItem__trackTitle')
    number_track = len(tracks)
    for i in range(len(tracks)):
        track_name = tracks[i].text
        track_list.append(track_name)
        track_link_list.append(tracks[i].get_attribute('href'))
    
    
    users = browser.find_elements_by_class_name('trackItem__username')
    for i in range(len(users)):
        if users[i].get_attribute('href'):
            user_link = users[i].get_attribute('href')
            user_name_list.append(users[i].text)
        else: 
            user_name_list.append(users[i].text)
            user_link = 'None'
        user_link_list.append(user_link)
        
    time.sleep(6)
    #Thêm vào playcount
    playcounts = browser.find_elements_by_class_name('trackItem__additional')
    for i in range(len(users)):
        if playcounts[i].text == 'Not available in Viet Nam':
            empty.append(i)
            playcount = None
        else:
            if len(playcounts[i].text) > 10:
                playcount = None
            else:
                playcount = playcounts[i].text
        playcount_list.append(playcount)
    
    track_durations = browser.find_elements_by_class_name('playbackTimeline__duration')
    for i in range(len(users)):
        if i in empty:
            duration_list.append(None)
        if track_durations[0].text:
            track_duration = str(track_durations[0].text.split('\n')[1])
        else:
            track_duration = None
        duration_list.append(track_duration)
        browser.find_elements_by_class_name('skipControl__next')[0].click()
        time.sleep(PAUSE_TIME)    
        
    file_write(track_file, track_list, track_link_list, user_name_list, user_link_list, playcount_list, duration_list)
    
    time.sleep(3)
    try:
        playlist_name = browser.find_element_by_class_name('soundTitle__title').text
    except NoSuchElementException:
        playlist_name = browser.find_element_by_class_name('fullHero__titleTextLineBig').text
    try:
        playlist_owner_name = browser.find_element_by_class_name('userBadge__usernameLink').text
        playlist_owner_url = browser.find_element_by_class_name('userBadge__usernameLink').get_attribute('href')
        playlist_likes = browser.find_element_by_class_name('sc-ministats-likes').text.split('\n')[1]
        playlist_posted_time = \
            browser.find_element_by_class_name('relativeTime').get_attribute('datetime').split('T')[0]

    except NoSuchElementException:
        playlist_owner_name = ''
        playlist_owner_url = ''
        playlist_likes = ''
        playlist_posted_time = ''
    f_playlist = open(playlist_file, 'a', encoding='utf-8')
    f_playlist.write(f'{playlist_name}\t{playlist_owner_name}\t{playlist_owner_url}\t{number_track}\t{playlist_posted_time}\t{playlist_likes}\n')
    f_playlist.close()

In [24]:
# link = 'https://soundcloud.com/randy-gerard-c-r/sets/study-electronic-chill'
# start = time.time()
# get_playlist_info('playlist_temp.csv', 'track_temp.csv', browser, link)
# print(time.time() - start)

42.94608449935913


In [31]:
# f = open('track.csv', 'w', encoding='utf-8')
# f.write('Track name\tUser name\tUser link\tPlay count\tDuration\n')
# f.close()

# f = open('playlist.csv', 'w', encoding='utf-8')
# f.write('playlist_name\tplaylist_owner_name\tplaylist_owner_url\tnumber_track\tplaylist_posted_time\tplaylist_likes\n')
# f.close()

In [33]:
# #Run
# file = open('link_playlists.txt', 'r')
# start = time.time()
# t = 1
# for link in file:
#         get_playlist_info('playlist.csv', 'track.csv', browser, link)
# file.close()
# print(time.time() - start)

8836.687911987305


- Lấy ngẫu nhiên 2000 user link từ file track.csv để khai thác thông tin user 

In [21]:
# f = open('track.csv', 'r', encoding='utf-8')
# jobs = f.readlines()
# test = random.choices(jobs[1:], k=2000)
# f.close()

In [22]:
# with open('link_user.csv', 'w', encoding='utf-8') as f:
#     for i in test:
#         user_link = i.split('\t')[3]
#         f.write(f'{user_link}\n')
# f.close()

- Hàm lấy thông tin từ user_link và lưu vào file .csv:

In [74]:
def get_user_info(filename, browser, link):
    user_link = link.replace('\n', '')
    SLEEP_TIME = 1
    
    #truy cập link:
    browser.get(link)
    time.sleep(SLEEP_TIME)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #Lấy thông tin:
    try:
        user_name = browser.find_elements_by_class_name('profileHeaderInfo__userName')[0].text.split('Pro')[0]
        follower = browser.find_elements_by_class_name('infoStats__value')[0].text
        following = browser.find_elements_by_class_name('infoStats__value')[1].text
        track_count = browser.find_elements_by_class_name('infoStats__value')[2].text
    except NoSuchElementException:
        user_name = None
        follower = None
        following = None
        track_count = None
    try:
        star = browser.find_element_by_class_name('profileHeaderInfo__premiumIndicator').text
        user_type = star
    except NoSuchElementException:
        user_type = 'Basic'
    f_playlist = open(filename, 'a', encoding='utf-8')
    f_playlist.write(f'{user_name}\t{user_link}\t{user_type}\t{follower}\t{following}\t{track_count}\n')
    

In [80]:
# #Thiếu dữ liệu 1191 (bd từ 1) do user k tồn tại
# f = open('link_user.csv', 'r', encoding='utf-8')
# start = time.time()
# for line in f:
#         get_user_info('user1.csv', browser, line)
# f.close()
# print(time.time() - start)

1141.0282909870148


In [175]:
data_tuples = list(zip(track_list, user_list, user_links, playcount_list, duration_list))
df = pd.DataFrame(data_tuples, columns=['Track','User name', 'User link', 'Playcount', 'Duration'])
# df.reset_index(inplace=True)
# df = df.rename(columns = {'index':'TrackID'})
df.to_csv("test.csv", encoding='utf-8', index = None)
df.tail(15)

,Track,User name,User link,Playcount,Duration
166,"Rvssian, Lil Mosey, Lil Tjay - Only The Team (...",Rvssian,https://soundcloud.com/rvssianofficial,15M,2:34
167,John Mayer - Slow Dancing In A Burning Room (D...,DECADON,https://soundcloud.com/decadon,210K,5:02
168,Disclosure - Latch (DJ Premier Remix),pmrrecords,https://soundcloud.com/pmrrecords,3.3M,3:26
169,Flume - Rushing Back (feat. Vera Blue) [Pardi ...,Pardi,https://soundcloud.com/pardiofficial,"5,627",3:32
170,The Less I Know the Crave You (A L E J A N D R...,A L E J A N D R O,https://soundcloud.com/alejandro-sdv,35.2K,3:56
171,Stroking,Arthur Young,https://soundcloud.com/arthuryoungofficial,407,3:56
172,DJ Affilination - Leftovers,King Records,https://soundcloud.com/minnesota-king-records,16K,3:06
173,pronouncedyea feat. David Blake - World On Fire,pronouncedyea,https://soundcloud.com/pronouncedyea,39.3K,3:06
174,Two Feet - Go Fuck Yourself (Kayfluxx Dub Remake),Kayfluxx,https://soundcloud.com/kayfluxx,821,3:15
175,I Met A Boy - Winona Oak(Kayfluxx Remix),Kayfluxx,https://soundcloud.com/kayfluxx,23.4K,3:04


In [228]:
link = 'https://soundcloud.com/digitalstreams/sets/saxxyhouse'
browser.get(link)
scrollDownAllTheWay(browser)

True

### Tiền xử lý:

- Tách tên ca sĩ và tên track trong track_list:

In [40]:
a_string = 'How Hard I Try ft. James Hersey'
len(a_string.split('-'))
# if a_string.split('-')[1]:
#     print("duma")

1

### Đọc dữ liệu từ file csv vào dataframe

## 2/ Tiền xử lý:

In [ ]:
        #Tiền xử lý:
#         if len(i_track.text.split('-')) > 1:
#             singer = i_track.text.split('-')[0]
#             track = i_track.text.split('-')[1]
#         else: 
#             singer = 0
#             track = i_track.text
#         singer_list.append(singer)